In [2]:
import pandas as pd
import numpy as np
import win32com.client
import time

In [47]:
file = "C:/Users\lukec\OneDrive - University of Cambridge\Projects\PhD\Data\IHS\Ethylene\ETHYLENE FROM WIDE RANGE NAPHTHA, MAXIMUM ETHYLENE, FRONT END DEMETHANIZER - Germany.xlsx"
footer = 11
headerLines = 5
colSep = 6

In [52]:
def refresh_excel_file(file:str, pause=False):
    # Refresh Excel file to have numbers for formula cells
    xlapp = win32com.client.DispatchEx("Excel.Application")
    wb = xlapp.Workbooks.Open(file)
    wb.RefreshAll()
    if pause: time.sleep(pause)
    wb.Save()
    wb.Close()
    xlapp.Quit()
    return file

In [167]:
def extract_properties(df):
    properties = list(map(lambda x: x.strip(), df.loc[2][0].split(",")))+['Name: '+df.loc[1][0]]
    properties = dict(map(lambda x: x.split(": "), properties))
    return properties

def add_property_columns(df, properties):
    print(list(properties.values()))
    df[list(properties.keys())] = list(properties.values())
    return df

def excel_multilayer_table_extract(df, dataType='Attribute', header=0, colStart=0, colEnd=-1, properties=False):
    data = df[header:][df.columns[colStart:colEnd]]
    data.columns = [dataType] + list(data.iloc[0][1:].values)
    data.columns = list(map(lambda x: 'Unit' if x is np.nan else x, data.columns.values))
    data.dropna(subset=[data.columns[0]], inplace=True)

    typesInd = [0] + list(np.where(data.index[1:] - data.index[:-1] != 1)[0]+1)
    data['Type'] = list(map(lambda x: data[dataType].values[typesInd[np.sum(np.array(typesInd) < x+1)-1]], range(len(data))))
    data.dropna(subset=data.columns[1:-1], how='all', inplace=True)
    data.reset_index(drop=True, inplace=True)
    if properties: data = add_property_columns(data, properties)
    return data

In [171]:
# Import sheet
sheet = pd.read_excel(refresh_excel_file(file),skipfooter=footer)
properties = extract_properties(sheet)
materials = excel_multilayer_table_extract(sheet, dataType='Material', header=headerLines, colEnd=colSep,
                                           properties=properties)
cost = excel_multilayer_table_extract(sheet, dataType='Cost', header=headerLines, colStart=colSep+1,
                                      properties=properties)

['Germany', 'ETHYLENE', '1087', '135.83 ¢/KG', '2M-1622', '1.06 (1US $ = 0.85)', '2021 Q3', '2014', 'ETHYLENE FROM WIDE RANGE NAPHTHA, MAXIMUM ETHYLENE, FRONT END DEMETHANIZER']


In [127]:
from pandas_xlsx_tables import df_to_xlsx_table
df_to_xlsx_table(cost, "Costs")

In [ ]:
#!pip install pivottablejs
# from pivottablejs import pivot_ui
# pivot_ui(cost,outfile_path='pivottablejs.html')

# Extract from multiline file

In [ ]:
import pyarrow

In [66]:
append = True
in_materials = 'C:/Users\lshc3\Documents\IHS_data\ethylene\materials.feather'
in_products = 'C:/Users\lshc3\Documents\IHS_data\ethylene\products.feather'
out_materials = 'C:/Users\lshc3\Documents\IHS_data\materials.feather'
out_products = 'C:/Users\lshc3\Documents\IHS_data\products.feather'

In [67]:
def del_empty_cols(df): return df.dropna(axis=1, how='all').reset_index(drop=True)

class ProcessData(object):
    def __init__(self):
        self.filePath = "C:/Users\lshc3\Documents\IHS_data/all_products_metric/1-1008.xlsx"
        self.data = del_empty_cols(pd.read_excel(self.filePath,header=6,skipfooter=1)).dropna(axis=0, how='all')
        self.products = del_empty_cols(self.data.loc[self.data['Type']=='Product'])
        self.materials = del_empty_cols(self.data.loc[self.data['Type']!='Product'])
    def format_materials(self):
        merged = pd.merge(self.materials, self.products, how='left', on=['Code'])
        merged['Unit_conv'] = ['MM ']*len(merged)+merged['Unit']+['/yr']*len(merged)
        merged['Uncertainty'] = [np.nan]*len(merged)
        merged['Provenance'] = ['IHS PEP']*len(merged)
        shortList = ['Code', 'Data Version_x', 'Name_x', 'Process_x', 'Research Year_x','Geography_x','Base Capacity (MM unit/year)','Unit_conv','Name_y','Unit Consumption','Consumption Unit','Uncertainty','Provenance']
        longList = shortList + [i for i in merged.columns if i not in shortList+['Geography_y','Data Version_y','Research Year_y','Process_y','Unit']]
        rename_dict = {'Data Version_x': 'Data Version', 'Name_x':'Source', 'Process_x':'Target', 'Research Year_x':'Research Year',
       'Geography_x':'Geography', 'Base Capacity (MM unit/year)':'Plant capacity', 'Unit_conv':'Capacity unit', 'Name_y':'Product',
       'Unit Consumption':'Value', 'Consumption Unit':'Value unit', 'Type_x':'Source type', 'Variable Cost_x':'Source cost',
       'Investment (MM US$)':'Product Investment (MM US$)', 'Type_y':'Product Type', 'Variable Cost_y':'Product variable cost', 'Fixed Costs':'Product fixed costs', 'Overhead + Tax, Ins.':'Product Overhead + Tax, Ins.', 'Plant Cash Cost ':'Product Plant Cash Cost', 'Depreciation':'Product Depreciation',
       'Plant Gate Costs ':'Product Plant Gate Costs', 'G&A, Sales, Res.':'Product G&A, Sales, Res.','ROI (15%)':'Product ROI (15%)'}
        return merged[longList].rename(columns=rename_dict)

In [68]:
data = ProcessData()
if append:
    pd.read_feather(in_materials).append(data.format_materials()).reset_index(drop=True).to_feather(out_materials)
    pd.read_feather(in_products).append(data.products).reset_index(drop=True).to_feather(out_products)
else:
    data.format_materials().to_feather(out_materials)
    data.products.to_feather(out_products)

## Interacting

In [44]:
import ipywidgets as ipw

In [ ]:
selectors = [ipw.SelectMultiple(options=data.products[i].unique(),rows=10,description=i, disabled=False) for i in ['Name', 'Process', 'Geography', 'Research Year']]

In [47]:
i = 'Name'
sel = ipw.SelectMultiple(options=data.products[i].unique(),rows=10,description=i, disabled=False)

In [16]:
data.products

,Code,Geography,Data Version,Research Year,Process,Unit,Base Capacity (MM unit/year),Investment (MM US$),Type,Name,...,Fixed Costs,"Overhead + Tax, Ins.",Plant Cash Cost,Depreciation,Plant Gate Costs,"G&A, Sales, Res.",Production Cost,ROI (15%),Product Value,Product Price
0,1M-981,"United States, Gulf Coast",2021 Q3,2002.0,ETHYLENE BY THE UOP/HYDRO METHANOL TO OLEFINS ...,KG,400.068470,884.905431,Product,ETHYLENE,...,5.893464,6.625514,34.643831,22.118850,56.762680,2.781679,59.544360,33.178274,92.722634,100.420560
1,2M-981,Germany,2021 Q3,2002.0,ETHYLENE BY THE UOP/HYDRO METHANOL TO OLEFINS ...,KG,400.068470,937.999757,Product,ETHYLENE,...,6.556605,7.251622,90.496201,23.445981,113.942182,4.611685,118.553867,35.168971,153.722838,135.826800
2,3M-981,Japan,2021 Q3,2002.0,ETHYLENE BY THE UOP/HYDRO METHANOL TO OLEFINS ...,KG,400.068470,778.716780,Product,ETHYLENE,...,5.141040,5.321030,106.393440,19.464588,125.858028,4.795513,130.653541,29.196881,159.850422,100.354422
3,4M-981,"China, Shanghai",2021 Q3,2002.0,ETHYLENE BY THE UOP/HYDRO METHANOL TO OLEFINS ...,KG,400.068470,584.037585,Product,ETHYLENE,...,3.742141,3.454213,80.661070,14.598441,95.259511,3.623418,98.882928,21.897661,120.780589,97.532505
4,1M-1595,"United States, Gulf Coast",2021 Q3,2014.0,ETHYLENE FROM 100% ETHANE,KG,1499.998216,2603.121527,Product,ETHYLENE,...,5.280392,5.692002,32.993959,17.354164,50.348123,2.362249,52.710372,26.031246,78.741618,100.420560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,4M-1602,"China, Shanghai",2021 Q3,2014.0,METHANOL TO OLEFINS BY THE DMTO-II PROCESS,KG,298.010187,394.934144,Product,ETHYLENE,...,3.570702,3.305171,104.954207,13.252371,118.206578,1.394799,119.601377,19.878556,139.479933,97.532505
240,1M-1603,"United States, Gulf Coast",2021 Q3,2014.0,METHANOL TO OLEFINS BY UOP ADVANCED MTO PROCESS,KG,288.938340,604.837579,Product,ETHYLENE,...,6.383301,6.666587,41.930773,20.933102,62.863875,0.952157,63.816032,31.399653,95.215685,100.420560
241,2M-1603,Germany,2021 Q3,2014.0,METHANOL TO OLEFINS BY UOP ADVANCED MTO PROCESS,KG,288.938340,641.127834,Product,ETHYLENE,...,7.194883,7.383074,104.769784,22.189088,126.958872,1.618611,128.577483,33.283632,161.861116,135.826800
242,3M-1603,Japan,2021 Q3,2014.0,METHANOL TO OLEFINS BY UOP ADVANCED MTO PROCESS,KG,288.938340,532.257070,Product,ETHYLENE,...,5.554709,5.286335,129.071071,18.421130,147.492201,1.768928,149.261130,27.631695,176.892824,100.354422


In [20]:
merged = pd.merge(data.materials, data.products, how='left', on=['Code'])

In [31]:
merged['Unit_conv'] = ['MM ']*len(merged)+merged['Unit']+['/yr']*len(merged)
merged['Uncertainty'] = [np.nan]*len(merged)
merged['Provenance'] = ['IHS PEP']*len(merged)

In [50]:
shortList = ['Code', 'Data Version_x', 'Name_x', 'Process_x', 'Research Year_x','Geography_x','Base Capacity (MM unit/year)','Unit_conv','Name_y','Unit Consumption','Consumption Unit','Uncertainty','Provenance']
longList = shortList + [i for i in merged.columns if i not in shortList+['Geography_y','Data Version_y','Research Year_y','Process_y','Unit']]

In [52]:
rename_dict = {'Data Version_x': 'Data Version', 'Name_x':'Source', 'Process_x':'Target', 'Research Year_x':'Research Year',
       'Geography_x':'Geography', 'Base Capacity (MM unit/year)':'Plant capacity', 'Unit_conv':'Capacity unit', 'Name_y':'Product',
       'Unit Consumption':'Value', 'Consumption Unit':'Value unit', 'Type_x':'Source type', 'Variable Cost_x':'Source cost',
       'Investment (MM US$)':'Product Investment (MM US$)', 'Type_y':'Product Type', 'Variable Cost_y':'Product variable cost', 'Fixed Costs':'Product fixed costs', 'Overhead + Tax, Ins.':'Product Overhead + Tax, Ins.', 'Plant Cash Cost ':'Product Plant Cash Cost', 'Depreciation':'Product Depreciation',
       'Plant Gate Costs ':'Product Plant Gate Costs', 'G&A, Sales, Res.':'Product G&A, Sales, Res.','ROI (15%)':'Product ROI (15%)'}

In [54]:
merged[longList].rename(columns=rename_dict)

,Code,Data Version,Source,Target,Research Year,Geography,Plant capacity,Capacity unit,Product,Value,...,Product fixed costs,"Product Overhead + Tax, Ins.",Product Plant Cash Cost,Product Depreciation,Product Plant Gate Costs,"Product G&A, Sales, Res.",Production Cost,Product ROI (15%),Product Value,Product Price
0,1M-981,2021 Q3,CATALYST,ETHYLENE BY THE UOP/HYDRO METHANOL TO OLEFINS ...,2002.0,"United States, Gulf Coast",400.06847,MM KG/yr,ETHYLENE,NaN,...,5.893464,6.625514,34.643831,22.118850,56.76268,2.781679,59.544360,33.178274,92.722634,100.420560
1,1M-981,2021 Q3,CAUSTIC SODA (50%),ETHYLENE BY THE UOP/HYDRO METHANOL TO OLEFINS ...,2002.0,"United States, Gulf Coast",400.06847,MM KG/yr,ETHYLENE,0.006260,...,5.893464,6.625514,34.643831,22.118850,56.76268,2.781679,59.544360,33.178274,92.722634,100.420560
2,1M-981,2021 Q3,"METHANOL, CRUDE",ETHYLENE BY THE UOP/HYDRO METHANOL TO OLEFINS ...,2002.0,"United States, Gulf Coast",400.06847,MM KG/yr,ETHYLENE,5.941383,...,5.893464,6.625514,34.643831,22.118850,56.76268,2.781679,59.544360,33.178274,92.722634,100.420560
3,1M-981,2021 Q3,C4-C5 MIXTURE,ETHYLENE BY THE UOP/HYDRO METHANOL TO OLEFINS ...,2002.0,"United States, Gulf Coast",400.06847,MM KG/yr,ETHYLENE,-0.436700,...,5.893464,6.625514,34.643831,22.118850,56.76268,2.781679,59.544360,33.178274,92.722634,100.420560
4,1M-981,2021 Q3,ETHANE-RICH GAS,ETHYLENE BY THE UOP/HYDRO METHANOL TO OLEFINS ...,2002.0,"United States, Gulf Coast",400.06847,MM KG/yr,ETHYLENE,-0.031340,...,5.893464,6.625514,34.643831,22.118850,56.76268,2.781679,59.544360,33.178274,92.722634,100.420560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2917,4M-1603,2021 Q3,PROPANE-RICH GAS,METHANOL TO OLEFINS BY UOP ADVANCED MTO PROCESS,2014.0,"China, Shanghai",288.93834,MM KG/yr,ETHYLENE,-0.027350,...,4.008685,3.580103,94.094853,13.815847,107.91070,1.299338,109.210038,20.723771,129.933810,97.532505
2918,4M-1603,2021 Q3,"PROPYLENE, CHEM GRADE",METHANOL TO OLEFINS BY UOP ADVANCED MTO PROCESS,2014.0,"China, Shanghai",288.93834,MM KG/yr,ETHYLENE,-1.111190,...,4.008685,3.580103,94.094853,13.815847,107.91070,1.299338,109.210038,20.723771,129.933810,97.532505
2919,4M-1603,2021 Q3,COOLING WATER,METHANOL TO OLEFINS BY UOP ADVANCED MTO PROCESS,2014.0,"China, Shanghai",288.93834,MM KG/yr,ETHYLENE,262.880116,...,4.008685,3.580103,94.094853,13.815847,107.91070,1.299338,109.210038,20.723771,129.933810,97.532505
2920,4M-1603,2021 Q3,ELECTRICITY,METHANOL TO OLEFINS BY UOP ADVANCED MTO PROCESS,2014.0,"China, Shanghai",288.93834,MM KG/yr,ETHYLENE,108.026508,...,4.008685,3.580103,94.094853,13.815847,107.91070,1.299338,109.210038,20.723771,129.933810,97.532505


In [37]:
merged['Type_x', 'Unit Cost', 'Cost Unit', 'Variable Cost_x', 'Investment (MM US$)', 'Type_y', 'Variable Cost_y','Fixed Costs','Overhead + Tax, Ins.']

,Code,Geography_x,Data Version_x,Research Year_x,Process_x,Type_x,Name_x,Unit Cost,Cost Unit,Unit Consumption,...,Depreciation,Plant Gate Costs,"G&A, Sales, Res.",Production Cost,ROI (15%),Product Value,Product Price,Unit_conv,Uncertainty,Provenance
0,1M-981,"United States, Gulf Coast",2021 Q3,2002.0,ETHYLENE BY THE UOP/HYDRO METHANOL TO OLEFINS ...,Raw Material,CATALYST,NaN,NaN,NaN,...,22.118850,56.76268,2.781679,59.544360,33.178274,92.722634,100.420560,MM KG/yr,NaN,IHS PEP
1,1M-981,"United States, Gulf Coast",2021 Q3,2002.0,ETHYLENE BY THE UOP/HYDRO METHANOL TO OLEFINS ...,Raw Material,CAUSTIC SODA (50%),54.013254,KG,0.006260,...,22.118850,56.76268,2.781679,59.544360,33.178274,92.722634,100.420560,MM KG/yr,NaN,IHS PEP
2,1M-981,"United States, Gulf Coast",2021 Q3,2002.0,ETHYLENE BY THE UOP/HYDRO METHANOL TO OLEFINS ...,Raw Material,"METHANOL, CRUDE",36.162973,KG,5.941383,...,22.118850,56.76268,2.781679,59.544360,33.178274,92.722634,100.420560,MM KG/yr,NaN,IHS PEP
3,1M-981,"United States, Gulf Coast",2021 Q3,2002.0,ETHYLENE BY THE UOP/HYDRO METHANOL TO OLEFINS ...,By-Product,C4-C5 MIXTURE,66.866204,KG,-0.436700,...,22.118850,56.76268,2.781679,59.544360,33.178274,92.722634,100.420560,MM KG/yr,NaN,IHS PEP
4,1M-981,"United States, Gulf Coast",2021 Q3,2002.0,ETHYLENE BY THE UOP/HYDRO METHANOL TO OLEFINS ...,By-Product,ETHANE-RICH GAS,25.110652,KG,-0.031340,...,22.118850,56.76268,2.781679,59.544360,33.178274,92.722634,100.420560,MM KG/yr,NaN,IHS PEP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2917,4M-1603,"China, Shanghai",2021 Q3,2014.0,METHANOL TO OLEFINS BY UOP ADVANCED MTO PROCESS,By-Product,PROPANE-RICH GAS,62.236497,KG,-0.027350,...,13.815847,107.91070,1.299338,109.210038,20.723771,129.933810,97.532505,MM KG/yr,NaN,IHS PEP
2918,4M-1603,"China, Shanghai",2021 Q3,2014.0,METHANOL TO OLEFINS BY UOP ADVANCED MTO PROCESS,By-Product,"PROPYLENE, CHEM GRADE",104.168419,KG,-1.111190,...,13.815847,107.91070,1.299338,109.210038,20.723771,129.933810,97.532505,MM KG/yr,NaN,IHS PEP
2919,4M-1603,"China, Shanghai",2021 Q3,2014.0,METHANOL TO OLEFINS BY UOP ADVANCED MTO PROCESS,Utilities,COOLING WATER,4.596596,M3,262.880116,...,13.815847,107.91070,1.299338,109.210038,20.723771,129.933810,97.532505,MM KG/yr,NaN,IHS PEP
2920,4M-1603,"China, Shanghai",2021 Q3,2014.0,METHANOL TO OLEFINS BY UOP ADVANCED MTO PROCESS,Utilities,ELECTRICITY,10.970000,KWH,108.026508,...,13.815847,107.91070,1.299338,109.210038,20.723771,129.933810,97.532505,MM KG/yr,NaN,IHS PEP


In [49]:
[i for i in merged.columns if i not in shortList+['Geography_y','Data Version_y','Research Year_y','Process_y','Unit']]

['Type_x',
 'Unit Cost',
 'Cost Unit',
 'Variable Cost_x',
 'Investment (MM US$)',
 'Type_y',
 'Variable Cost_y',
 'Fixed Costs',
 'Overhead + Tax, Ins.',
 'Plant Cash Cost ',
 'Depreciation',
 'Plant Gate Costs ',
 'G&A, Sales, Res.',
 'Production Cost ',
 'ROI (15%)',
 'Product Value ',
 'Product Price ']

In [48]:
np.where(data.materials.columns == data.products.columns)

ValueError: Lengths must match to compare